In [1]:
import os
import ccxt
import configparser
import vectorbt as vbt
from datetime import datetime
import pytz
import json
import numpy as np
import talib as ta
import pandas as pd
import schedule
import time
import warnings
import requests
import decimal
warnings.filterwarnings('ignore')

#pd.set_option('display.max_rows', None)

#### Global configuration

In [2]:
config = configparser.ConfigParser()
config.read('key.ini')
# --------------------------------------------------
conf_robot_name = 'ActionZone'
conf_api_key = config['key']['apikey'] 
conf_api_secret = config['key']['secretkey']
conf_symbol = 'BTC-PERP' # trade symbol
conf_timeframe = '1m' # only support timeframe: 1m, 3m, 5m, 15m, 1h
conf_max_candles = 100 # total candles to be loaded from exchange, max is 1,000
conf_riskpertrade = 0.0001 # 1%
conf_position_size_limit = 1 # max position size to allow to trade
conf_leverage = 20
#conf_max_retry = 10 # max retry to get data
# --------------------------------------------------
conf_line_token = config['key']['line_token']
conf_line_url = 'https://notify-api.line.me/api/notify'
conf_line_headers = {'content-type':'application/x-www-form-urlencoded','Authorization':'Bearer '+conf_line_token}
# --------------------------------------------------
exchange = ccxt.ftx({
    'apiKey' : conf_api_key ,
    'secret' : conf_api_secret ,
    'enableRateLimit': True,
    'option' : {'defaultType' : 'future', 'adjustForTimeDifference': True}
})
exchange.headers = {
    'FTX-SUBACCOUNT': 'testAPI',
}

LOGFILE = f'{conf_robot_name}_log.txt' 

response = exchange.private_post_account_leverage({'leverage': conf_leverage,}) 
print(exchange.fetchStatus())

{'status': 'ok', 'updated': None, 'eta': None, 'url': None}


In [3]:
def line_notify(msg):
    r = requests.post(conf_line_url, headers=conf_line_headers, data ={'message':str(msg)})
    r.text

def get_time():    
    now = datetime.now()
    formatted_date = now.strftime("%d/%m/%Y %H:%M:%S")
    return formatted_date

def get_wallet():
    try:
        wallet = exchange.privateGetWalletBalances()['result']
        return wallet
    except:
        return None

def get_cash():
    try:
        wallet = get_wallet()
        for t in wallet:
            if t['coin'] in  ['USD']:
                cash = float(t['availableWithoutBorrow'])
        return cash
    except:
        return None
    
def get_position():
    try:
        res = exchange.fetchPositions()
        for sym in res:
            if sym['info']['future']==conf_symbol:
                f_pos = sym['info']
        return f_pos
    except Exception as e:
        log(str(e),LOGFILE)
        return None     
    
def get_price_digit(symbol): # นับ digit จาก fetch ticker ล่าสุด
    try:
        res =exchange.fetch_ticker(symbol)# ['info']['minProvideSize']
        price_step = float(res['info']['priceIncrement'])
        str_digit = str(price_step)
        count_digit = len(str_digit.split('.')[1])
        return count_digit
    except :
        pass
    
def get_size_digit(symbol): # นับ digit จาก fetch ticker ล่าสุด
    try:
        res =exchange.fetch_ticker(symbol)# ['info']['minProvideSize']
        size_step = float(res['info']['sizeIncrement'])
        last_price = float(res['info']['last'])
        min_provide = size_step * last_price
        str_digit = str(size_step)
        count_digit = len(str_digit.split('.')[1])

        return count_digit
    except :
        pass    
 
def get_minimum_size(symbol): ##จะไม่ได้ต้องเซ็ต fetch minimum จาก exchange กัน error
    res = exchange.fetch_ticker(symbol)['info']#['sizeIncrement']
    minimum_size = float(res['sizeIncrement'])
    last_bid = float(res['bid'])
    min_value =minimum_size * last_bid
    return float(minimum_size)

conf_min_size = get_minimum_size(conf_symbol)           

In [4]:
def reset_trade_log():
    latest_log['symbol'] = ''
    latest_log['entry_time'] = 0 
    latest_log['entry_price'] = 0.0 # average open price
    latest_log['position_side'] = '' # LONG or SHORT
    latest_log['position_amount'] = 0.0 # position size
    latest_log['stop_loss'] = 0.0 # stop loss price
    latest_log['take_profit'] = 0.0 # take profit price 
    
    
def load_trades_log(df):
    last_position = check_positions()
    if last_position == 1:
        f_pos = get_position()
        latest_log['symbol'] =  f_pos['future']
        latest_log['entry_time'] = df['timestamp'].iloc[-1]
        latest_log['entry_price'] =  float(f_pos['entryPrice'])  
        latest_log['position_side'] = f_pos['side'] 
        latest_log['position_amount'] =  float(f_pos['size'])
        latest_log['stop_loss'] = float(f_pos['entryPrice']) - Cal_SLdistance(df)  
        latest_log['take_profit'] = 0.0
    
    elif last_position == -1:
        f_pos = get_position()
        latest_log['symbol'] =  f_pos['future']
        latest_log['entry_time'] = df['timestamp'].iloc[-1]
        latest_log['entry_price'] =  float(f_pos['entryPrice'])  
        latest_log['position_side'] = f_pos['side'] 
        latest_log['position_amount'] =  float(f_pos['size'])
        latest_log['stop_loss'] = float(f_pos['entryPrice']) + Cal_SLdistance(df)  
        latest_log['take_profit'] = 0.0
    else: # Postion == 0
        reset_trade_log()
    return latest_log    

In [5]:
def Cal_SLdistance(df, vol_multiply = 1.2):
    vol = ta.STDDEV(df.close,30).ewm(alpha=0.96).mean()
    sl_distance = vol * vol_multiply
    return sl_distance.iloc[-1]

def Cal_TPdistance(df, vol_multiply = 1.2):
    vol = ta.STDDEV(df.close,30).ewm(alpha=0.96).mean()
    tp_distance = vol * vol_multiply
    return tp_distance.iloc[-1]

def Cal_Size(df, conf_riskpertrade = conf_riskpertrade):
    cash =get_cash()
    if cash != None:
        riskpertrade = conf_riskpertrade * cash
        size = (riskpertrade / Cal_SLdistance(df)) 
        if size > conf_position_size_limit:
            size = conf_position_size_limit
        elif size < conf_min_size:
            size = conf_min_size    
    return round(size, get_size_digit(conf_symbol))          

In [6]:
def check_positions():
    res = exchange.private_get_positions()['result']   
    for symbol in res:
        if symbol['future'] in conf_symbol:
            netsize = float(symbol['netSize'])
            if netsize > 0:
                positions = 1 # Have Long Positions
            elif netsize < 0:
                positions = -1 # Have Short Positions
            else :
                positions = 0 # No Positions
            return positions  

def place_longposition(df):    
    size = Cal_Size(df)
    exchange.create_order(conf_symbol, 'market', 'buy', size)
    
    line_notify(f"Open Long {conf_symbol} at {df.close.iloc[-1]}, \
                Stoploss : {df.close.iloc[-1] - Cal_SLdistance(df)}, Position Size : {size}")

def place_shortposition(df):    
    size = Cal_Size(df)
    exchange.create_order(conf_symbol, 'market', 'sell', size)
    
    line_notify(f"Open Short {conf_symbol} at {df.close.iloc[-1]}, \
                Stoploss : {df.close.iloc[-1] + Cal_SLdistance(df)}, Position Size : {size}")
    
def close_positions():    
    netsize = float(get_position()['netSize'])
    print(netsize)
    if netsize > 0:
        exchange.create_order(symbol = conf_symbol
                                        , type = 'market'
                                        , side = 'sell'
                                        , amount = abs(netsize)
                                        , params ={'conditionalOrdersOnly':True})
        
        msg = '------ Close Long ------'
        line_notify(msg) 
        
    elif netsize < 0:  
        exchange.create_order(symbol = conf_symbol
                                        , type = 'market'
                                        , side = 'buy'
                                        , amount = abs(netsize)
                                        , params ={'conditionalOrdersOnly':True})
        
        msg = '------ Close Short ------'
        line_notify(msg)
        
    else :
        print('No positions to close')


In [7]:
def is_new_bar(prev_bar_time, cur_bar_time):
    if cur_bar_time > prev_bar_time:
        return True
    else:
        return False  
    
def fetch_data(symbols = conf_symbol, timeframe = conf_timeframe, limit = conf_max_candles):
    try_count = 0 
    try:        
        bars = exchange.fetch_ohlcv(symbols, timeframe, limit = limit)
        df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except :
        print('LOAD DATA ERROR ')
        pass
        return pd.DataFrame()

def strategy(df):    
    if (not df.empty) :
        
        df['ema1'] = ta.EMA(df.close, 12)
        df['ema2'] = ta.EMA(df.close, 26)
        
        df['LongEntries'] = (df['ema1'] > df['ema2']) & (df['ema1'].shift(1) < df['ema2']) & (df.close > df['ema1'])
        df['LongExit'] = df.close < df['ema2']   
        df['ShortEntries'] = (df['ema1'] < df['ema2']) & (df['ema1'].shift(1) > df['ema2']) & (df.close < df['ema1'])
        df['ShortExit'] = df.close > df['ema2'] 
        return df
    
    else:
        print('Cant Fetch Data' )
        return df

In [8]:
def append_log(cur_bar_str, last_position, last_price):
    if last_position ==1:
        diff = last_price - latest_log['entry_price']
        pnl = diff * latest_log['position_amount']
        
    if last_position ==-1:
        diff = latest_log['entry_price'] - last_price
        pnl = diff * latest_log['position_amount']
        
    line = '{},{},{},{},{},{},{},{}\n'.format(
        latest_log['symbol'], latest_log['entry_time'], latest_log['entry_price'], latest_log['position_side']
        , latest_log['position_amount'], cur_bar_str, last_price, pnl
    )
    f = open(LOGFILE, "a")
    f.write(line + "\n")
    f.close()

    
def writes_trades_log():
    line = 'symbol, entry_time, entry_price, position_side, position_amount, exit_time, exit_price, pnl'
    f = open(LOGFILE, 'w')
    f.write(line + "\n")
    f.close()     
    
    

In [9]:
def trading():
    global prev_bar
    now_ts = int(exchange.milliseconds()*1000) #
    
    if prev_bar != 0:
        prev_bar_str = datetime.utcfromtimestamp(int(prev_bar/1000))
    else:
        prev_bar_str= []
        
    df_raw = fetch_data()
    df = strategy(df_raw) # add fetchingdata to strategy
    
    if  not df.empty:
        LongEntries = df['LongEntries'].iloc[-1] #[last_row_index]
        LongExit = df['LongExit'].iloc[-1]   
        ShortEntries = df['ShortEntries'].iloc[-1]
        ShortExit = df['ShortExit'].iloc[-1]
        
        print('Long : ',LongEntries,LongExit )    
        print('SHORT: ',ShortEntries,ShortExit )
        
        cur_bar_str =   df.timestamp.iloc[-1]
        cur_bar_time = int(cur_bar_str.timestamp()*1000) #

        new_bar_cond = is_new_bar(prev_bar,cur_bar_time)
        
        last_position = check_positions()
        last_price = df['close'].iloc[-1]
        
        
        print(f" Latest Position : {latest_log}")     

        print('----------------------------------------')
        print(f"Fetching new bars for  {prev_bar_str} : {cur_bar_str} : , Exchange Mileseconds :{now_ts} {cur_bar_time}")

        print(f"EMA12 : {df['ema1'].iloc[-1]}, EMA26 : {df['ema2'].iloc[-1]}  Last Price {last_price}")
        print(f"Position : {last_position}")
        
        if (prev_bar != cur_bar_time) :
            print('NEW BAR')
            if last_position == 0: 

                if LongEntries == True:    
                    place_longposition(df)                               
                    load_trades_log(df) 
                    print("------ Open Long ------")

                elif ShortEntries == True: 
                    place_shortposition(df)  
                    load_trades_log(df)                                                                            
                    print("------ Open Short ------")

                else :
                    print('No Positions and Signals')

            elif last_position == 1: 

                if LongExit == True :
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions() 
                    load_trades_log(df)      
                    print("------ Exit Long ------")

                elif  (latest_log['take_profit'] != 0.0) and (last_price >= latest_log['take_profit']):
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions() 
                    load_trades_log(df)          
                    print('TAKE PROFIT Long')

                elif  (latest_log['stop_loss'] != 0.0) and (last_price <= latest_log['stop_loss']):
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions()
                    load_trades_log(df)
                    print('STOPLOSS Long')

                elif ShortEntries == True: # Reverse Signal
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions()
                    load_trades_log(df)
                        
                    place_shortposition(df) 
                    load_trades_log(df)
                    print("------ Open Short ------")
                else:
                    print('CHECK PNL')
                    print('Have Long Positions, No signal')
                    print(latest_log)


            elif last_position== -1: # Have Short Positions 

                if ShortExit == True :
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions()
                    load_trades_log(df)                
                    print("------ Exit Short ------")

                elif  (latest_log['take_profit'] != 0.0) and (last_price <= latest_log['take_profit']):
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions()
                    load_trades_log(df)                
                    print('TAKE PROFIT Short')
                    
                elif  (latest_log['stop_loss'] != 0.0) and (last_price >= latest_log['stop_loss']):
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions()
                    load_trades_log(df) 
                    print('STOPLOSS Short')                                       

                elif LongEntries == True: # Reverse Signal
                    append_log(cur_bar_str, last_position, last_price)
                    close_positions()
                    load_trades_log(df)
                        
                    place_longposition(df)
                    load_trades_log(df)
                    print("------ Open Long ------")

                else:
                        print('CHECK PNL')
                        print('Have Short Positions, No signal')
                        print(latest_log)

        else:
            print('SAMEBAR')
            
        prev_bar = cur_bar_time
        print('-'*50)
        
    else:
        print('Can Not get DataFrame')
        pass

In [10]:
writes_trades_log()

In [ ]:
from IPython.display import clear_output
latest_log = {}
latest_log['symbol'] = ''
latest_log['entry_time'] = 0 
latest_log['entry_price'] = 0.0 # average open price
latest_log['position_side'] = '' # LONG or SHORT
latest_log['position_amount'] = 0.0 # position size
latest_log['stop_loss'] = 0.0 # stop loss price
latest_log['take_profit'] = 0.0 # take profit price
count = 0
prev_bar = 0
while True:
    trading()
    count += 1
    time.sleep(10)
    if count == 1000:
        count = 0
        clear_output(wait=True) 

Long :  False False
SHORT:  False True
 Latest Position : {'symbol': '', 'entry_time': 0, 'entry_price': 0.0, 'position_side': '', 'position_amount': 0.0, 'stop_loss': 0.0, 'take_profit': 0.0}
----------------------------------------
Fetching new bars for  2022-03-21 05:16:00 : 2022-03-21 05:16:00 : , Exchange Mileseconds :1647839856009000 1647839760000
EMA12 : 40913.17627743431, EMA26 : 40899.45687382279  Last Price 40920.0
Position : 0
SAMEBAR
--------------------------------------------------
Long :  False False
SHORT:  False True
 Latest Position : {'symbol': '', 'entry_time': 0, 'entry_price': 0.0, 'position_side': '', 'position_amount': 0.0, 'stop_loss': 0.0, 'take_profit': 0.0}
----------------------------------------
Fetching new bars for  2022-03-21 05:16:00 : 2022-03-21 05:16:00 : , Exchange Mileseconds :1647839866652000 1647839760000
EMA12 : 40913.17627743431, EMA26 : 40899.45687382279  Last Price 40920.0
Position : 0
SAMEBAR
-------------------------------------------------